In [ ]:
"""
Created on Mon Aug 18 17:37 2025

Prepare a domain cfg and a param melt for cavities all closed for my experiments

@author: Clara Burgard
"""

In [68]:
import xarray as xr
from tqdm.notebook import tqdm
import numpy as np
from cdo import Cdo
from datetime import date


In [49]:
cdo = Cdo()
print('this is CDO version %s'%(cdo.version()))

this is CDO version 1.9.9rc1


In [2]:
inputpath_raw2 = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/raw/'
inputpath_interim = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/interim/'
outputpath_newmask = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/processed/'

Prepare the domain cfg for all cavities closed

In [3]:
domain_cfg = xr.open_dataset(inputpath_raw2 + 'eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights/eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights_domain_cfg.nc')

In [4]:
domain_cfg_new = domain_cfg.copy()
for vvar in ['bottom_level', 'top_level', 'isf_draft', 'bathy_metry', 'mask_opensea', 'mask_csgrpglo']:
    domain_cfg_new[vvar] = domain_cfg_new[vvar].where(domain_cfg['isf_draft'] == 0, 0)

In [5]:
domain_cfg_new.to_netcdf(outputpath_newmask + 'eORCA1.4.3_OpenSeas_CloseAllCav_ModStraights_domain_cfg.nc')

Prepare the injection of meltwater

In [6]:
ds_isfNEMO = xr.open_dataset(inputpath_interim + 'masks_for_eORCA1.nc')
file_Justine = xr.open_dataset(inputpath_interim + 'Mask_Iceshelf_IMBIE2_v2_with_zmin_zmax_isfconc.nc').drop('quantile')

In [7]:
new_ds_isfNEMO = ds_isfNEMO.copy()

In [8]:
# Choose (by eye or given a criterion, which ice shelves will have open cavities)
ID_open_list = []

In [9]:
# Divide the ref file into "open" and "closed" cavities
file_Justine_closed = file_Justine

In [10]:
# Prepare mask of only open cavities 
mask_isf_open = ds_isfNEMO['mask_isf'] * np.nan

# Prepare mask of only closed cavities 
mask_isf_closed = ds_isfNEMO['mask_isf'].where(np.isnan(mask_isf_open))

In [11]:
# create mask_isf_open and mask_isf_closed
new_ds_isfNEMO['mask_isf_open'] = mask_isf_open
new_ds_isfNEMO['mask_isf_open'].attrs['standard_name'] = 'mask of the ice shelves that are open'
new_ds_isfNEMO['mask_isf_open'].attrs['units'] = 'ID as given in Mask_Iceshelf_IMBIE2_v2.nc'

new_ds_isfNEMO['mask_isf_closed'] = mask_isf_closed
new_ds_isfNEMO['mask_isf_closed'].attrs['standard_name'] = 'mask of the ice shelves that are closed'
new_ds_isfNEMO['mask_isf_closed'].attrs['units'] = 'ID as given in Mask_Iceshelf_IMBIE2_v2.nc'

IDENTIFY FRONT

In [50]:
isfmask_nn = cdo.remapnn(inputpath_interim + 'NEMO_grid_withbnds.nc',input=file_Justine['Iceshelf_extrap'], returnArray='Iceshelf_extrap')
isfmask = xr.DataArray(data=isfmask_nn, dims=['y','x'])

In [29]:
# Create a mask discriminating between land, ocean and ice shelf
mask_0_1_2 = new_ds_isfNEMO['mask_isf'] * 2
mask_0_1_2 = mask_0_1_2.where(domain_cfg['bathy_metry'] != 0, 400) # land
mask_0_1_2 = mask_0_1_2.where(domain_cfg['bathy_metry'] == 0, 0) # ocean
mask_0_1_2 = mask_0_1_2.where(domain_cfg['isf_draft'] == 0, 200) # ice shelf

In [51]:
# Create a mask of the extrapolated domains and only keep the domains corresponding to the closed cavities
extrap_domains_closed = isfmask.copy()

for kisf in ID_open_list:
    mask_0_1_2_closed = mask_0_1_2.where(new_ds_isfNEMO['mask_isf'] != kisf, 400)
    extrap_domains_closed = extrap_domains_closed.where(extrap_domains_closed != kisf, np.nan)

In [52]:
# set all ice shelves to 300
mask_front0 = mask_0_1_2.where((mask_0_1_2 == 0) | (mask_0_1_2 == 400), 300).copy()

In [53]:
mask_front = mask_front0.copy()
# check all directions and set points at border between ocean and ice shelf (300-0) to 500
mask_front = mask_front.where((mask_front0.shift(x=-1)-mask_front0)!=300,500)
mask_front = mask_front.where((mask_front0.shift(x=1)-mask_front0)!=300,500)
mask_front = mask_front.where((mask_front0.shift(y=-1)-mask_front0)!=300,500)
mask_front = mask_front.where((mask_front0.shift(y=1)-mask_front0)!=300,500)
# cut out all front points
mask_front = mask_front.where(mask_front==500)
# set the ice shelf number
mask_front = mask_front.where(mask_front!=500,extrap_domains_closed)

For the ice shelves not resolved at all in eORCA1

In [55]:
# define the domains that have not an associated front yet
extrap_domains_double_closed = extrap_domains_closed.copy()

for kisf in file_Justine_closed.ID:
    if (mask_front == kisf).astype(int).sum() > 0:
        #print(kisf.values)
        extrap_domains_double_closed = extrap_domains_double_closed.where(extrap_domains_double_closed != kisf, np.nan).drop('ID')

In [56]:
# set all ice shelves to land to have a delimitation of the whole contour of Antarctica
mask_0_2 = mask_0_1_2.where(mask_0_1_2 != 200, 2)
mask_0_2 = mask_0_2.where(mask_0_2 != 400, 2)

mask_front_new = mask_0_2.copy()

# check all directions and set points at border between ocean and land(300-0) to 500
mask_front_new = mask_front_new.where((mask_0_2.shift(x=-1)-mask_0_2) <= 0,5)
mask_front_new = mask_front_new.where((mask_0_2.shift(x=1)-mask_0_2) <= 0,5)
mask_front_new = mask_front_new.where((mask_0_2.shift(y=-1)-mask_0_2) <= 0,5)
mask_front_new = mask_front_new.where((mask_0_2.shift(y=1)-mask_0_2) <= 0,5)
# cut out all front points south of the y = 90 latitude and in the ocean domain
mask_front_new = mask_front_new.where((mask_front_new == 5) & (mask_front_new.y < 90) & (mask_0_2 == 0))
# set the ice shelf number of the extrapolated domain
mask_front_new = mask_front_new.where(mask_front_new != 5,extrap_domains_double_closed)

In [57]:
# combining the masks of the fronts (mask_front and mask_front_new)
mask_front_all = mask_front.copy()
for iid in file_Justine_closed.ID:
    if (new_ds_isfNEMO['mask_isf_closed'] == iid).astype(int).sum().values == 0: # if the ice shelf is not resolved in your grid, add the mask front inferred for them
        #print(iid.values, file_Justine_closed['NAME'].sel(ID=iid).values, (mask_front_new == iid).astype(int).sum().values)
        mask_front_all = mask_front_all.combine_first(mask_front_new.where(mask_front_new == iid))

In [59]:
mask_front_file = xr.Dataset()
mask_front_file['mask_front'] = mask_front_all
mask_front_file['ID'] = file_Justine['ID']
mask_front_file.to_netcdf(inputpath_interim + 'mask_fronts_eORCA1_allISFclosed.nc')

Prepare the file to be eaten by NEMO

In [60]:
file_mask_eORCA1 = xr.open_dataset(inputpath_interim + 'masks_for_eORCA1.nc')

In [62]:
zmin_front = mask_front_all.copy()
zmax_front = mask_front_all.copy()

for id_closed in tqdm(file_Justine_closed.ID):
    zmin_front = zmin_front.where(mask_front_all != id_closed, file_Justine_closed['z_perc01'].sel(ID=id_closed)).drop('ID')
    zmax_front = zmax_front.where(mask_front_all != id_closed, file_Justine_closed['z_perc99'].sel(ID=id_closed)).drop('ID')

  0%|          | 0/133 [00:00<?, ?it/s]

In [63]:
cell_area = domain_cfg['e1t'] * domain_cfg['e2t']
melt_src = 'Davison' # options: 'Adusumilli','Rignot', 'Paolo', 'Davison'
melt_flux = file_Justine_closed['Melt'+melt_src]

In [64]:
melt_front = mask_front_all.copy()
for id_closed in tqdm(file_Justine_closed.ID):        
    cell_area_isf = cell_area.where(mask_front_all == id_closed)
    cell_area_isf_sum = cell_area.where(mask_front_all == id_closed).sum()
    
    if id_closed == 100:
        melt_front = melt_front.where(mask_front_all != id_closed, (melt_flux.sel(ID=101) + melt_flux.sel(ID=100)) * cell_area_isf / cell_area_isf_sum).drop('ID')
    elif id_closed == 88:
        melt_front = melt_front.where(mask_front_all != id_closed, (melt_flux.sel(ID=89) + melt_flux.sel(ID=88)) * cell_area_isf / cell_area_isf_sum).drop('ID')
    elif id_closed == 14:
        melt_front = melt_front.where(mask_front_all != id_closed, (melt_flux.sel(ID=14) + melt_flux.sel(ID=15)) * cell_area_isf / cell_area_isf_sum).drop('ID')
    elif id_closed == 20:
        melt_front = melt_front.where(mask_front_all != id_closed, (melt_flux.sel(ID=20) + melt_flux.sel(ID=17) + melt_flux.sel(ID=18)) * cell_area_isf / cell_area_isf_sum).drop('ID')
    elif id_closed == 102:
        melt_front = melt_front.where(mask_front_all != id_closed, (melt_flux.sel(ID=102) + melt_flux.sel(ID=105) + melt_flux.sel(ID=107)) * cell_area_isf / cell_area_isf_sum).drop('ID')
    else:
        melt_front = melt_front.where(mask_front_all != id_closed, melt_flux.sel(ID=id_closed) * cell_area_isf / cell_area_isf_sum).drop('ID')


  0%|          | 0/133 [00:00<?, ?it/s]

In [65]:
zmin_front = zmin_front.where(melt_front > 0)
zmax_front = zmax_front.where(melt_front > 0)
melt_front = melt_front.where(melt_front > 0)

In [66]:
new_ds_isfNEMO['z_min'] = zmin_front
new_ds_isfNEMO['z_min'].attrs['standard_name'] = 'most shallow point of the ice-shelf draft indicated at the ice-shelf front'
new_ds_isfNEMO['z_min'].attrs['units'] = 'm below sea level'

new_ds_isfNEMO['z_max'] = zmax_front
new_ds_isfNEMO['z_max'].attrs['standard_name'] = 'deepest point of the ice-shelf draft indicated at the ice-shelf front'
new_ds_isfNEMO['z_max'].attrs['units'] = 'm below sea level'

new_ds_isfNEMO['melt_isf_closed'] = melt_front
new_ds_isfNEMO['melt_isf_closed'].attrs['standard_name'] = 'melt flux from '+melt_src+' distributed at the ice-shelf front'
new_ds_isfNEMO['melt_isf_closed'].attrs['units'] = 'Gt/yr'

In [69]:
new_ds_isfNEMO.attrs=dict(Source='Based on IMBIE2 (Mask_Iceshelf_IMBIE2_v2.nc), melt from '+melt_src,
                      Compatibility='The masked variables are compatible with eORCA1.4.3_OpenSeas_CloseAllCav_ModStraights_domain_cfg.nc',
                      Creator='C. Burgard  ('+date.today().strftime("%b-%d-%Y")+')')

In [70]:
new_ds_isfNEMO.to_netcdf(inputpath_interim + 'masks_for_eORCA1_allISFclosed.nc') 

Prepare the file for NEMO

In [71]:
inputpath_raw='/data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/raw/'
file_IPSL = xr.open_dataset(inputpath_raw + 'eORCA1.4.2_runoff-icb_DaiTrenberth_Depoorter.nc')